# CartoonGAN : Generative Adversarial Networks for Photo Cartoonization
  

# Abstract(논문 개요)
- 사진을 카툰화해주는 생성적 적대적 신경망
- 분야 : computer vision, computer graphics
- 최근들어 이미지를 스타일링하는 것이 유행하고 있다.
- 하지만, 만족스러운 결과를 만들어내지는 못했다.
  - 이유 1. 만화는 높은 수준의 단순화 및 추상화와 함께 독특한 특징들을 가지고 있다.
  - 이유 2. 만화는 경계선이 깔끔하고, 색이 부드럽고, 상대적으로 단순한 텍스쳐를 가지는 경향이 있다. 따라서, 기존 방식인 텍스쳐를 기반으로 한 손실함수로는 상당한 어려움이 존재한다.
- CartoonGAN은 만화 형식으로 스타일링해주는 GAN 모델이다.
- 훈련 데이터 : 짝지어 지지 않은 사진, 만화 이미지
- loss
  - 1. semantic content loss(의미론적 콘텐츠 손실)
  - 사진과 만화 사이의 상당한 스타일 변화를 다루기 위해 VGG 네트워크의 고수준의 feature map에서 희귀성을 위한 정규화를 만들어낸다.
  - 2. 깔끔한 경계선을 지키기 위한 경계선을 강조하는 적대적 loss
- 초기화 단계, 타켓 매니폴드(target manifold)를 위한 컨볼루젼 네트워크를 향상시키는 방법 소개


# 1. Introduction
- 만화는 일상 생활에서 널리 활용되고 있다.
- 많은 유명한 만화 이미지는 실제 장면을 기반으로 만들어졌다.
- 실제 장면을 만화 스타일로 수작업하는 것은 매우 힘들고 상당한 예술적 스킬을 포함한다.
- 하지만, 현재 나와있는 소프트웨어나 알고리즘으로는 만족할만한 결과물이 나오지 않는다.
- 따라서 자동으로 실제 장면을 카툰화해주는 것이 생기면 아티스트에게도 도움이 되고, 시간 절약도 되고, 좀 더 창조적인 일에 집중할 수 있게 된다.
- 또한, 인스타그램, 포토샵과 같은 사진 편집 소프트웨어에도 유용한 도움이 된다.
- 예술적 방식으로 이미지를 스타일링 하는 방식은 비사실적 렌더링 분야에서 광범위하게 연구되었다.
- 기존의 방식은 특정 스타일에 대한 전용 알고리즘을 개발한다.
- 하지만, 각각의 아티스트를 모방하는 것은 상당한 노력이 필요하다.
- 최근에는 주어진 이미지를 기반으로 스타일링이 가능한 학습 기반 스타일 변화 방법이 상당한 주목을 받고 있다.
- 특히, 짝지어지지 않은 사진과 스타일된 이미지로부터 훈련된다는 독특한 특징을 가지고 있는 GAN 모델이.
- 학습 기반 스타일 변화 방법은 상당한 성공을 거두었지만 괜찮은 품질의 카툰화된 이미지를 만들어내는 것에는 실패했다.
  - 이유 1. 텍스쳐를 추가하는 것 대신에, 만화는 실제 모습과 비교하여 매우 단순하고 추상적이다.
  - 이유 2. 아티스트들의 다양한 스타일에도 불구하고, 만화는 다른 형태의 예술 작품과는 다르게 눈에띄는 공통적인 특성을 갖는다. - 경계선이 깔끔하고, 부드러운 색상 음영, 상대적으로 단순한 텍스쳐를 갖는다.
- 훈련 데이터 : 사진 데이터셋, 만화 이미지 데이터셋
- 훈련 데이터로부터 고 품질의 결과를 만들어 내기 쉽게 하기 위해서, 사진과 만화 두 데이터셋에서 일치하거나 짝지어지는 데이터를 필요로 하지 않는다.
- computer vision 알고리즘 관점으로부터, 만화 스타일링의 목표는 컨텐츠를 변경하지 않는 동안 여러 개의 사진을 여러 개의 만화로 매핑하는 것이다.
- 이 목표를 달성하기 위해서, 우리는 전용 GAN 기반 architecture를 사용할 것을 제안한다. 간단하면서도 효과적인 손실 함수 2개를 사용하면서.
- 이 논문의 중요한 핵심
  - 1. 짝지어지지 않은 훈련을 위한 사진, 만화 데이터셋 사용.
  - 최신 기법보다 더 좋은 품질의 카툰화 이미지 출력이 목표.
  - 개개인의 아티스트의 만화를 훈련에 사용하면, 우리의 도구는 그들의 스타일로 재생산해준다.
  - 2. GAN기반의 architecture에서 간단하지만 효과적인 2개의 손실 함수를 이용.
  - 생성 신경망에서, 사진과 만화 사이의 상당한 스타일 차이를 다루기 위해서 우리는 의미론적 손실을 VGG 네트워크의 고수준 feature map에서의 $l_{1}$ 희귀성 정규화(sparse reguralization)로 소개하겠다.
  - 판별자 네트워크에서, 우리는 깔끔한 경계선을 위해서, 경계선을 강조하는 적대적 손실을 제안한다.
  - 3. 현재 존재하는 방법들보다 훨씬 더 효과적인 방법이다.
  - 더 나아가서 타겟 매니폴드로 네트워크가 수렴되는 것을 향상시키기 위한 초기화 단계를 설명하겠다.

# 2. Related Work(관련 작업)
## 2.1 Non-photorealistic rendering(NPR, 비사실적 렌더링)
- 특정 만화 아티스트들의 스타일을 모방하기 위해서 자동적, 반자동적인 많은 비사실적 렌더링(NPR) 알고리즘들이 개발되었다.
- 일부 작품들은 간단한 음영만으로도 만화효과스러운 3D 모양을 렌더링했다.
- 이러한 "cel shading"(셀 셰이딩) 이라고 불리는 기술들은 아티스트들의 많은 시간을 아껴주고, 만화 비디오와 만화 영화를 만드는데 사용되었다.
- 이미지 필터링, 최적화 방법을 이용한 생성을 이용해서 만화 스타일을 흉내낸 이미지를 만드는 방법들은 많이 나와있다.
- 그러나, 존재하는 사진이나 비디오를 만화처럼 바꾸는 것은 엄청 어려운 일이다.
- 또한, 간단한 수학 공식을 이용해서는 풍부한 예술적 스타일을 포착하기 어렵다.
- 특히, 필터를 추가하거나 최적화 방법으로는 경계선을 깔끔하게 만드는 등과 같이 고차원의 추상적인 아티스트의 행동을 따라하는 건 어렵다.
- 이러한 결과를 개선시키기 위해서, 대체 방법으로 나온 것이 이미지와 비디오를 분할하는 것이다. 하지만, 사용자 상호작용이 필요하다.
- 초상화를 위해 개발된 전용 방법도 있다. 의미론적 분할(semantic segmentation)은 얼굴의 구성요소를 발견하는데서 자동적으로 유래되었다.
- 하지만, 그러한 방법은 일반적인 이미지에 대해서는 잘 대응되지 않는다.


## 2.2 Stylization with neural networks
- CNN은 많은 computer vision 문제들을 해결하는데 많은 관심을 받았다.
- 각각의 스타일에 상당한 노력이 필요한 비사실적 렌더링(NPR) 알고리즘을 개발하는 것 대신에, style transfer가 활발히 연구되고 있다.
- 짝을 이룬 스타일/비스타일 이미지를 필요로 하는 전통적인 style transfer 방식과는 달리, 최근의 연구에 따르면 객체 인식을 위해 훈련된 VGG 네트워크는 style transfer에서 매우 중요한 객체의 의미론적 특징을 추출하는 능력이 우수하다.
- 결과적으로, 짝을 이룬 이미지가 필요하지 않은 더 강력한 style transfer 방법이 개발되었다는 것이다.
- CNN을 기반으로 한 neural style transfer 방법은 스타일 이미지에서 콘텐츠 이미지로 스타일을 전송시켜준다.
- CNN을 기반으로 한 neural style transfer는 미리 훈련된 VGG 네트워크를 사용한다. 콘텐츠 이미지를 표현하고 결과 이미지를 최적화한다. 스타일 이미지의 텟트쳐 정보를 매칭하면서 콘텐츠 이미지의 콘텐츠를 유지하도록. 여기서 텍스쳐는 global gram matrix를 사용한다.

---
---
# 추가 개념 설명
## 비사실적 렌더링
- 컴퓨터 그래픽스의 한 영역. 사실적인 렌더링 이외의 표현. 
- 양식. 회화, 드로잉, 만화 같이 렌더링하는 것.

## 희소 표현(sparse representation)
- 대부분 0으로 표현되는 것. 종종 차원의 크기가 크다.

## 특성 교차
- 특성들을 조합하여 새로운 특성을 만들어내는 것
- 원래보다 차원의 크기가 커진다.
- 원래보다 자원을 더 많이 필요로 한다.

## 자원을 많이 차지할 때
- 가중치가 정확하게 0으로 떨어지도록 유도하는 것이 좋다.
- 가중치가 정확하게 0이 될 경우 모델에서 해당 특성을 삭제하게 되므로, 자원 소모량이 줄어들게 된다.

## 희소성을 위한 정규화 : $l_{1}, l_{2}$ 규제(정규화)
- 규제 : 가중치의 성장을 제한한다. 일반적인 패턴이 아닌 몇몇 독특하면서 희소한 패턴을 가지는 데이터(noise, outlier)로부터 영향을 적게 받겠다.
- $l_{1}$ 
  - 가중치의 절댓값에 패널티를 준다.
  - 관련성이 없거나 매우 낮은 특성의 가중치를 정확히 0으로 유도. 모델에서 해당 특성을 배제하는 데 도움이 된다.
- $l_{2}$
  - 가중치의 제곱값에 패널티를 준다.
  - 가중치를 0에 가깝게 유도하는 데 도움을 준다.
  - 선형 모델의 일반화를 항상 개선한다.
- 참고 : https://developers.google.com/machine-learning/crash-course/regularization-for-sparsity/l1-regularization?hl=ko

## 매니폴드(manifold)
- 매니폴드란 고차원 데이터가 있을 때, 고차원 데이터를 데이터 공간에 뿌리면 sample들을 잘 아우르는 subspace가 있을 것이라는 가정하에서 학습을 진행하는 방법.
- manifold는 고차원 데이터를 잘 표현한다. 즉, 데이터의 중요한 특징을 잘 발견한다는 뜻.
- 샘플이 집중적으로 놓인 원래 특징 공간보다 낮은 차원의 공간을 의미.
- 두 점의 거리 or 유사도를 보았을 때
- 근거리에서는 유클리디안(직선 거리)를 따른다.
- 원거리에서는 유클리디안을 따르지 않는다.
- 조그만 유클리디안 공간 조각들이 다닥다닥 붙어 이루어져서 전체적으로 보면 비유클리디안이 되는 공간을 의미한다.
- <img src="https://t1.daumcdn.net/tistoryfile/fs4/10_6_6_30_blog106301_attach_0_3.jpg?original" width=250>
- 참고 : https://markov.tistory.com/39

## 매니폴드 학습(manifold learning)
- 데이터(샘플)을 잘 아우르는 manifold를 찾게 되었다. -> 데이터에 핵심적인 feature를 잘 찾았다. -> manifold의 좌표를 조금씩 변경해가면서 데이터를 유의미하게 조금씩 변화시킬 수 있다.

## 셀 셰이딩(cel shading)
- 비사실적으로 묘사하는 그래픽 기법 중 하나로 손으로 그린 듯한 효과를 준다.
- 만화를 모방하기 위해 자주 사용하는 기법이다.

## Classification VS Localization(Detection) VS Segmentation
- Classification(분류) : input에 대해 하나의 label을 예측
- Localization/Detection(발견) : label을 예측 + 어디에 위치하는 지 정보를 제공. 물체가 있는 곳에 네모를 그리는 등
- Segmentation(분할) : 모든 픽셀의 label을 예측. 물체의 모양대로 테두리 선을 그림.
- <img src="https://miro.medium.com/max/700/1*cJ3oLJ2s8W_sCPmTuMgIlw.png">
- 참고 : https://medium.com/hyunjulie/1%ED%8E%B8-semantic-segmentation-%EC%B2%AB%EA%B1%B8%EC%9D%8C-4180367ec9cb

## Semantic Image Segmentation
- 목적 : 이미지에 있는 모든 픽셀을 해당되는 class로 분류하는 것.
- 이미지에 있는 모든 픽셀에 대한 예측을 하는 것이기 때문에 dense prediction이라고도 불린다.
- Semantic Image Segmentation은 같은 class의 instance를 구별하지 않는다. 즉, 사람 2명과 강아지 1마리가 있을 때 사람1, 사람2, 강아지라고 다른 사람이라고 구분하는 것이 아니라 사람, 사람, 강아지 라고 구분한다.

## Gram Matrix
- style transfer를 이해하기 위해서는 Gram Matrix에 대한 이해가 필요.
- style transfer는 Gram Matrix의 차이를 줄이는 방향으로 학습한다.
- texture들간의 rgb 공분산을 구하는 것.
- style transfer에서 content(사진)과 style(스타일)을 섞을 때 사용됨
- Gram Matrix의 역할 : 네트워크의 각 층의 style representation들을 서로 내적하여 구한다. -> 각각의 층의 Gram Matrix의 loss를 줄이는 방향으로 학습 -> content, style의 loss가 어느 지점에서 합의를 보게 될 것인지를 구해줌.
- 참고 : https://kyounju.tistory.com/3
- 참고 : picture 폴더 내의 필기 3개 캡쳐 이미지
